# Tomographic mapping notebook, with optimizations for minor phases  
__Written by Haixing Fang, Jon Wright and James Ball__  
__Date: 21/02/2025__

This notebook will try to reconstruct grain shapes and positions from the grain orientations you found in the first notebook.  
This notebook is optimised for weak and noisy minor phase data.  
This notebook (and the tomo route in general) works best for low levels of deformation.  
If it doesn't seem to work well, try the point-by-point route instead!

In [ ]:
import os

os.environ['OMP_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

In [ ]:
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())

In [ ]:
# this cell is tagged with 'parameters'
# to view the tag, select the cell, then find the settings gear icon (right or left sidebar) and look for Cell Tags

# python environment stuff
PYTHONPATH = setup_ImageD11_from_git( ) # ( os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

# dataset file to import
dset_path = 'si_cube_test/processed/Si_cube/Si_cube_S3DXRD_nt_moves_dty/Si_cube_S3DXRD_nt_moves_dty_dataset.h5'

# phase names to look at
major_phase_strs = ['Fe']
minor_phase_str = 'Au'

# peak filtration parameters
# whether to filter out major phase peaks before trying to build sinograms
remove_major_phase_peaks = True
major_phase_cf_dstol = 0.005
minor_phase_cf_frac = 0.9975
minor_phase_cf_dstol = 0.005

# If the sinograms are only half-sinograms (we scanned dty across half the sample rather than the full sample), set the below to true:
is_half_scan = False
# If we did halfmask, choose the radius to mask in the centre of the reconstruction (normally hot pixels)
halfmask_radius = 25

# assign peaks to the grains with hkl tolerance peak_assign_tol
peak_assign_tol = 0.05

# tolerance for building sinograms from assigned peaks
hkltol = 0.25

# We can optionally correct each row of the sinogram by the ring current of that rotation
# This helps remove artifacts in the reconstruction
correct_sinos_with_ring_current = True

first_tmap_cutoff_level = 0.4

# how many iterations for Astra reconstruction?
niter = 500

second_tmap_cutoff_level = 0.5

# filter out grains with more than grain_too_many_px pixels in the label map
# this normally indicates a dodgy reconstruction for this grain
# only really applies if the grains are very small!
grain_too_many_px = 10

In [ ]:
import concurrent.futures

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from tqdm.notebook import tqdm

from ImageD11 import cImageD11
from ImageD11.peakselect import select_ring_peaks_by_intensity, remove_peaks_from_phases
import ImageD11.sinograms.dataset
from ImageD11.sinograms.sinogram import GrainSinogram, write_h5, read_h5, get_2d_peaks_from_4d_peaks
from ImageD11.sinograms.tensor_map import TensorMap
import ImageD11.nbGui.nb_utils as utils

%matplotlib ipympl

# Load data
## Dataset

In [ ]:
ds = ImageD11.sinograms.dataset.load(dset_path)
print(ds)

## Phases
If the parameter file was a json, we can access the unit cells via `ds.phases.unitcells`

In [ ]:
ds.phases = ds.get_phases_from_disk()
ds.phases.unitcells

Now we need to decide which are our 'major' phases, which we will remove from our peaks before indexing.

In [ ]:
major_phase_unitcells = [ds.phases.unitcells[mps] for mps in major_phase_strs]
print(*major_phase_unitcells)
minor_phase_unitcell = ds.phases.unitcells[minor_phase_str]
print(minor_phase_unitcell)

# Peaks

In [ ]:
cf_4d = ds.get_cf_4d()
ds.update_colfile_pars(cf_4d)  # computes geometry, needed for filtration
for major_phase_unitcell in major_phase_unitcells:
    major_phase_unitcell.makerings(cf_4d.ds.max())
minor_phase_unitcell.makerings(cf_4d.ds.max())

# Grains

In [ ]:
grains = ds.get_grains_from_disk(minor_phase_str)
grainsinos = [GrainSinogram(g, ds) for g in grains]
utils.get_rgbs_for_grains(grains)
print(f"{len(grains)} grains imported")

In [ ]:
utils.plot_all_ipfs(grains)

## Major phase grains
We inherit the reconstruction parameters you used for your major phase grains

In [ ]:
major_phase_grainsinos = read_h5(ds.grainsfile, ds, major_phase_strs[0])  # pick one of the major phases
whole_sample_mask = major_phase_grainsinos[0].recon_mask
shift = major_phase_grainsinos[0].recon_shift
pad = major_phase_grainsinos[0].recon_pad
y0 = major_phase_grainsinos[0].recon_y0
print(shift, y0, pad)
if is_half_scan:
    ds.correct_bins_for_half_scan(y0)

# Filtration
## (optional) remove major phase peaks

In [ ]:
if remove_major_phase_peaks:
    cf_4d = remove_peaks_from_phases(cf_4d, major_phase_cf_dstol, major_phase_unitcells)

## Filter by intensity
Now we are filtering our peaks (`cf_4d`) to select only the strongest ones .
This time, we don't clip via `dsmax`, because we need the higher-angle projections for complete sinograms.  
We first filter the peaks in $d^{*}$ to keep only those close to the predicted peaks from the unit cell.  
We then sort our peaks by intensity, and take a certain intensity-weighted fraction of them.  
`dstol`: The tolerance in $d^{*}$ between a peak and a predicted reflection.  
`frac`: The intensity fraction: `frac=0.9` keeps 90% of the peak intensity. We recommend that you choose a value close to the 'elbow' of the plot.

In [ ]:
cf_strong = select_ring_peaks_by_intensity(cf_4d, frac=minor_phase_cf_frac, dstol=minor_phase_cf_dstol, dsmax=cf_4d.ds.max(), ucell=minor_phase_unitcell, doplot=0.5)

In [ ]:
minor_phase_unitcell.makerings(cf_strong.ds.max())
skip = 1  # we can skip peaks to speed up plotting if needed
fig, ax = plt.subplots(figsize=(10, 5), constrained_layout=True)
ax.plot(cf_4d.ds[::skip], cf_4d.sum_intensity[::skip],',', label='cf_4d',c='blue')
ax.plot(cf_strong.ds[::skip], cf_strong.sum_intensity[::skip],',', label='minor phase',c='orange')
ax.vlines(minor_phase_unitcell.ringds, 1e3, 3e3, color='red', label=minor_phase_unitcell.name)
for inc, major_phase_unitcell in enumerate(major_phase_unitcells):
    ax.vlines(major_phase_unitcell.ringds, 3e3*(3**inc), 3e3*(2**inc)*3, color=plt.color_sequences['tab20'][inc], label=major_phase_unitcell.name)
ax.set(xlabel=r'$d^{*}~(\AA^{-1})$', ylabel='Intensity', yscale='log', title='Peak filtration')
ax.legend(loc='upper right')
plt.show()

# Peak assignment

In [ ]:
utils.assign_peaks_to_grains(grains, cf_strong, peak_assign_tol)
for grain_label, g in enumerate(grains):
    g.gid = grain_label
    g.npks_4d = np.sum(cf_strong.grain_id == grain_label)

In [ ]:
utils.plot_grain_sinograms(grains, cf_strong, min(len(grains), 25))

# Grain position refinement

In [ ]:
for grain_label, gs in enumerate(grainsinos):
    gs.update_lab_position_from_peaks(cf_strong, grain_label)

In [ ]:
# plt.style.use('dark_background')
fig, ax = plt.subplots(2,2, figsize=(12,12), layout='constrained')
a = ax.ravel()
x = [g.translation[0] for g in grains]
y = [g.translation[1] for g in grains]
s = [g.npks_4d/10 for g in grains]
a[0].scatter(y, x, c=[g.rgb_z for g in grains], s=s)
a[0].set(title='IPF color Z',  aspect='equal')
a[1].scatter(y, x, c=[g.rgb_y for g in grains], s=s)
a[1].set(title='IPF color Y', aspect='equal')
a[2].scatter(y, x, c=[g.rgb_x for g in grains], s=s)
a[2].set(title='IPF color X',  aspect='equal')
a[3].scatter(y, x, c=s)
a[3].set(title='Number of 4D peaks', aspect='equal')
fig.supxlabel("<- Sample y (transverse)")
fig.supylabel("Sample x (beam) ->")
fig.suptitle("Grain centre-of-mass positions from 4D peaks")
for a in ax.ravel():
    a.invert_xaxis()
plt.show()

# Grain sinogram generation

In [ ]:
gord, inds = get_2d_peaks_from_4d_peaks(ds.pk2d, cf_strong)
for grain_label, gs in enumerate(tqdm(grainsinos)):
    gs.prepare_peaks_from_4d(cf_strong, gord, inds, grain_label, hkltol)

In [ ]:
# ^ if you segmented frelon data for scanning, the above cell won't work, because we don't have a 2d peaks table
# you can instead try the below to prepare the peaks from the 2D peaks:

# cf_2d = ds.get_cf_2d_from_disk()
# ds.update_colfile_pars(cf_2d, phase_str)
# for grain_label, gs in enumerate(tqdm(grainsinos)):
#     gs.prepare_peaks_from_2d(cf_2d, grain_label, hkltol)

In [ ]:
for gs in tqdm(grainsinos):
    gs.build_sinogram()

In [ ]:
# optionally correct the halfmask:
if is_half_scan:
    for gs in grainsinos:
        gs.correct_halfmask()

In [ ]:
gs = grainsinos[0]
fig, ax = plt.subplots(figsize=(7, 7), layout='constrained')
ax.pcolormesh(gs.sinoangles, ds.ybincens, gs.ssino)
ax.set(xlabel=r'Projection angle ($\degree$)', ylabel='dty', title='Grain sinogram')
plt.show()

In [ ]:
# We can optionally correct each row of the sinogram by the ring current of that rotation
# This helps remove artifacts in the reconstruction
if correct_sinos_with_ring_current:
    ds.get_ring_current_per_scan()
    for gs in grainsinos:
        gs.correct_ring_current(is_half_scan=is_half_scan)

In [ ]:
gs = grainsinos[0]
fig, ax = plt.subplots(figsize=(7, 7), layout='constrained')
ax.pcolormesh(gs.sinoangles, ds.ybincens, gs.ssino)
ax.set(xlabel=r'Projection angle ($\degree$)', ylabel='dty', title='Grain sinogram')
plt.show()

# Grain shape reconstruction
## iradon
### First grain as example

In [ ]:
gs.update_recon_parameters(pad=pad, shift=shift, mask=whole_sample_mask, y0=y0)
gs.recon()
if is_half_scan:
    gs.mask_central_zingers("iradon", radius=halfmask_radius)
fig, axs = plt.subplots(1,2, figsize=(10,5), layout="constrained")
axs[0].pcolormesh(gs.sinoangles, ds.ybincens, gs.ssino)
axs[0].set(xlabel=r'Projection angle ($\degree$)', ylabel='dty', title='Grain sinogram')
axs[1].imshow(gs.recons["iradon"], vmin=0, origin="lower")
axs[1].set(title="ID11 iradon", xlabel="<-- Sample Y", ylabel="Sample X")
plt.show()

In [ ]:
for gs in grainsinos:
    gs.update_recon_parameters(pad=pad, shift=shift, mask=whole_sample_mask, y0=y0)

### Reconstruct all grains

In [ ]:
nthreads = cImageD11.cores_available()
with concurrent.futures.ThreadPoolExecutor(max_workers= max(1,nthreads-1)) as pool:
    for i in tqdm(pool.map(GrainSinogram.recon, grainsinos), total=len(grainsinos)):
        pass
if is_half_scan:
    for gs in grainsinos:
        gs.mask_central_zingers("iradon", radius=halfmask_radius)

### View outputs

In [ ]:
def update_frame(i):
    rec.set_array(grainsinos[i].recons["iradon"])
    sin = a[1].pcolormesh(grainsinos[i].sinoangles, ds.ybincens, grainsinos[i].ssino)
    fig.suptitle("Grain " + str(i))
    fig.canvas.draw()
frame_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(grains) - 1,
    step=1,
    description='Grain:'
)
fig, a = plt.subplots(1,2,figsize=(10,5), constrained_layout=True)
rec = a[0].imshow(grainsinos[0].recons["iradon"], vmin=0, origin="lower")
sin = a[1].pcolormesh(grainsinos[0].sinoangles, ds.ybincens, grainsinos[0].ssino)
a[0].set(xlabel="<-- Sample Y", ylabel="Sample X", title="iradon Reconstruction")
a[1].set(xlabel=r'Projection angle ($\degree$)', ylabel='dty', title='Grain sinogram')
widgets.interact(update_frame, i=frame_slider)
plt.show()

### Assemble TensorMap

In [ ]:
tensor_map = TensorMap.from_grainsinos(grainsinos, cutoff_level=first_tmap_cutoff_level)
tensor_map.plot("ipf_z")
tensor_map.plot("labels")
tensor_map.plot("intensity")

## ASTRA
We can clean up these reconstructions using an MLEM iterative recon.  
We will carry this out using ASTRA on the GPU on the SLURM cluster.  
The ASTRA EM_CUDA method will be used.

In [ ]:
for gs in grainsinos:
    gs.update_recon_parameters(pad=pad, shift=shift, mask=whole_sample_mask, niter=niter, y0=y0)
write_h5(ds.grainsfile, grainsinos, overwrite_grains=True, group_name=minor_phase_str)

In [ ]:
bash_script_path = utils.prepare_astra_bash(ds, ds.grainsfile, PYTHONPATH, group_name=minor_phase_str)
utils.slurm_submit_and_wait(bash_script_path, 10)

In [ ]:
grainsinos = read_h5(ds.grainsfile, ds, group_name=minor_phase_str)
for gs, g in zip(grainsinos, grains):
    gs.grain = g
    gs.ds = ds

### View outputs

In [ ]:
def update_frame(i):
    rec.set_array(grainsinos[i].recons["iradon"])
    sin = a[1].pcolormesh(grainsinos[i].sinoangles, ds.ybincens, grainsinos[i].ssino)
    fig.suptitle("Grain " + str(i))
    fig.canvas.draw()
frame_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(grains) - 1,
    step=1,
    description='Grain:'
)
fig, a = plt.subplots(1,2,figsize=(10,5), constrained_layout=True)
rec = a[0].imshow(grainsinos[0].recons["astra"], vmin=0, origin="lower")
sin = a[1].pcolormesh(grainsinos[0].sinoangles, ds.ybincens, grainsinos[0].ssino)
a[0].set(xlabel="<-- Sample Y", ylabel="Sample X", title="Astra MLEM Reconstruction")
a[1].set(xlabel=r'Projection angle ($\degree$)', ylabel='dty', title='Grain sinogram')
widgets.interact(update_frame, i=frame_slider)
plt.show()

### Assemble TensorMap

In [ ]:
tensor_map_astra = TensorMap.from_grainsinos(grainsinos, cutoff_level=second_tmap_cutoff_level, method="astra")
tensor_map_astra.plot("ipf_z")
tensor_map_astra.plot("labels")
tensor_map_astra.plot("intensity")

# Filter by reconstruction quality  
For weak grains, there will likely be many streaks, indicating a few grains have dodgy reconstructions and are probably not to be trusted.  
To fix this, we can count how many pixels in the grain labels array each grain has.  
It can be helpful to run this filtration more than once.  

In [ ]:
labels, counts = np.unique(tensor_map_astra["labels"], return_counts=True)
fig, ax = plt.subplots(constrained_layout=True, figsize=(6, 6))
ax.plot(labels[labels > 0], counts[labels > 0])
ax.set(xlabel='Grain ID', ylabel='Pixels in TensorMap')
plt.show()

Filter out grains with more than grain_too_many_px pixels in the label map.  
This normally indicates a dodgy reconstruction for this grain.  
Only really applies if the grains are very small!

In [ ]:
bad_gids = [int(label) for (label, count) in zip(labels, counts) if count > grain_too_many_px and label > 0]
print(f"{len(grainsinos)} grains before filtration")
grainsinos_clean = [gs for (inc, gs) in enumerate(grainsinos) if inc not in bad_gids]
grain_labels_clean = [inc for (inc, gs) in enumerate(grainsinos) if inc not in bad_gids]
print(f"{len(grainsinos_clean)} grains after filtration")
for gs, label in zip(grainsinos_clean, grain_labels_clean):
    gs.grain.gid = label

### Assemble cleaned TensorMap

In [ ]:
tensor_map_astra = TensorMap.from_grainsinos(grainsinos_clean, cutoff_level=second_tmap_cutoff_level, method="astra")
tensor_map_astra.plot("ipf_z")
tensor_map_astra.plot("labels")
tensor_map_astra.plot("intensity")

### Re-fit grain positions from reconstructions

In [ ]:
for gs in tqdm(grainsinos):
    gs.update_lab_position_from_recon()
for gs in grainsinos:
    gs.update_recon_parameters(y0=y0)

In [ ]:
# plt.style.use('dark_background')
fig, ax = plt.subplots(2,2, figsize=(12,12), layout='constrained')
a = ax.ravel()
x = [gs.grain.translation[0] for gs in grainsinos_clean]
y = [gs.grain.translation[1] for gs in grainsinos_clean]
s = [gs.grain.npks_4d/10 for gs in grainsinos_clean]
a[0].scatter(y, x, c=[gs.grain.rgb_z for gs in grainsinos_clean], s=s)
a[0].set(title='IPF color Z',  aspect='equal')
a[1].scatter(y, x, c=[gs.grain.rgb_y for gs in grainsinos_clean], s=s)
a[1].set(title='IPF color Y', aspect='equal')
a[2].scatter(y, x, c=[gs.grain.rgb_x for gs in grainsinos_clean], s=s)
a[2].set(title='IPF color X',  aspect='equal')
a[3].scatter(y, x, c=s)
a[3].set(title='Number of 4D peaks', aspect='equal')
fig.supxlabel("<- Sample y (transverse)")
fig.supylabel("Sample x (beam) ->")
fig.suptitle("Grain centre-of-mass positions from 4D peaks")
for a in ax.ravel():
    a.invert_xaxis()
plt.show()

# Export
## Grains

In [ ]:
write_h5(ds.grainsfile, grainsinos, overwrite_grains=True, group_name=minor_phase_str)

## TensorMap

In [ ]:
tensor_map_astra.to_h5(ds.grainsfile, h5group='TensorMap_' + minor_phase_str)

## XDMF

In [ ]:
tensor_map_astra.to_paraview(ds.grainsfile, h5group='TensorMap_' + minor_phase_str)

In [ ]:
ds.save()